#  Inert doublet model
Be sure to run [index_bash](../index_bash) first, in order to have all the required binaries

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
from hep import *
import pdg_series
#from indirectdirect import *
from multicurvefit import *
import time

Define functions to change from general basis to physical basis 

In [3]:
def phys_to_int(mH,mA,mHc,lambda_L,v):
    '''
    See arXiv:1003.3125
    '''
    mH2=mH*mH;mA2=mA*mA;mHc2=mHc*mHc;v2=v*v
    lambda_5=(mH2-mA2)/v2
    mu2=mH2-lambda_L*v2
    lambda_3=2.*(mHc2-mu2)/v2
    lambda_4=-lambda_3-lambda_5+2*lambda_L
    return mu2,lambda_3,lambda_4,lambda_5

def int_to_phys(mu2,lambda_3,lambda_4,lambda_5,v):
    '''
    See arXiv:1003.3125
    '''
    v2=v*v
    tachyons=False
    mHc2=mu2+lambda_3*v2/2.
    if mHc2<0: tachyons=True
    mH2=mu2+(lambda_3+lambda_4+lambda_5)*v2/2.
    if mH2<0: tachyons=True
    mA2=mu2+(lambda_3+lambda_4-lambda_5)*v2/2.
    if mA2<0: tachyons=True
    if tachyons: print "Warning: Tachyionic masses"
    return np.sqrt(np.abs(np.array([mH2,mA2,mHc2]))),(lambda_3+lambda_4+lambda_5)/2 

### SARAH implementation
Based in [Scotogenic model implementation](https://github.com/restrepo/Scotogenic) by Avelino Vicente. Model files in the [SARAH/Models/SimplifiedDM/IDM](../SARAH/Models/SimplifiedDM/IDM) folder of this repo. We use below the python [hep](./hep.py) module to automalically manage input/output SARAH-Toolbox files (in a similar way to SSP)

In [4]:
a=hep(MODEL='SimplifiedDMIDM')

`a-object` is an object with many attributes and methods. Use the tab to explore them. Some of them are
* a.Series: [pandas](http://pandas.pydata.org/) Series object with the "relevant" variables 
* a.LHA: Input LesHouces file as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runSPheno() -> a.LHA_out: return LHA output files as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runmicromegas() -> a.runSPheno() -> Updated the `a-object`  with micrOMEGAS "relevant" output

## Scan 
As in [arXiv:1605.01129](https://arxiv.org/abs/1605.01129) but for region of interest

In [9]:
npoints=2
df=pd.DataFrame()
ipt=pd.Series({'MH0':40,'MA0':701,'MHC':701,'lambda_L':0.1})
#SPHENO SETTINGS:
a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
dm_masses=np.linspace(60,75,npoints)
for MH0 in dm_masses:
    ps=pd.Series()
    if np.where(dm_masses==MH0)[0][0]%10==0: #find the index of the array entry
        print np.where(dm_masses==MH0)[0][0]
        
    ipt.MH0=MH0
    if ipt.MH0<70:
        mhcmin=70
    else:
        mhcmin=ipt.MH0
        
    ipt.MHC=np.random.uniform(mhcmin,700)
    ipt.MA0=np.random.uniform(110,700)
    ipt.lambda_L=np.exp( np.random.uniform(log(1E-5),log(2)) )*np.random.choice([1,-1])
    mu2,lambda_3,lambda_4,lambda_5=phys_to_int(ipt.MH0,ipt.MA0,ipt.MHC,ipt.lambda_L,a.vev)
    a.LHA.blocks['MINPAR'][3]='%0.8E       #lambda3' %lambda_3
    a.LHA.blocks['MINPAR'][4]='%0.8E       #lambda4' %lambda_4
    a.LHA.blocks['MINPAR'][5]='%0.8E       #lambda5' %lambda_5
    a.LHA.blocks['MINPAR'][6]='%0.8E       #mu2' %mu2
    mo=a.runmicromegas(Direct_Detection=True)
    a.branchings(a.LHA_out.decays,min_pdg=25) #Creates a.Br_names pandas Series
    ps['lambda_L']=ipt.lambda_L
    ps=ps.append(pd.Series({'MH0':a.LHA_out.blocks['MASS'][35],\
                                        'MA0':a.LHA_out.blocks['MASS'][36],\
                                        'MHc':a.LHA_out.blocks['MASS'][37]}))
    ps=ps.append(a.Series)
    ps=ps.append(a.Br_names)
    df=df.append(ps,ignore_index=True)

0


In [10]:
ps

lambda_L                    -7.703573e-02
MA0                          1.130849e+02
MH0                          7.500000e+01
MHc                          1.365736e+02
GFFermiconstant              1.166370e-05
Zbosonpolemass               9.118870e+01
alphasMZSMMSbar              1.187000e-01
lambda1                      1.300000e-01
lambda2                      0.000000e+00
lambda3                      2.756997e-01
lambda4                     -3.116144e-01
lambda5                     -1.181568e-01
mbmbSMMSbar                  4.180000e+00
mtaupole                     1.776690e+00
mtoppole                     1.735000e+02
mu2                          1.029526e+04
Omega_h2                     7.510000e-02
proton_SI                    3.566000e-08
proton_SD                    0.000000e+00
neutron_SI                   3.637000e-08
neutron_SD                   0.000000e+00
sigmav                       4.060000e-26
ID_br:~etR,~etR -> d3 D3     6.250000e-01
ID_br:~etR,~etR -> Wp Wm     1.740

In [7]:
df

,GFFermiconstant,"ID_br:~etR,~etR -> Wp Wm","ID_br:~etR,~etR -> Z Z","ID_br:~etR,~etR -> d3 D3","ID_br:~etR,~etR -> e3 E3","ID_br:~etR,~etR -> g g","ID_br:~etR,~etR -> u2 U2",MA0,MH0,MHc,...,mtaupole,mtoppole,mu2,neutron_SD,neutron_SI,proton_SD,proton_SI,sigmav,"ID_br:~etR,~etR -> A A","ID_br:~etR,~etR -> d2 D2"
0,0.000012,0.884,0.1080,0.00682,0.000413,0.0005,0.000634,627.140555,59.999997,500.723032,...,1.77669,173.5,3581.01721,0,9.330000e-13,0,9.147000e-13,1.500000e-27,NaN,NaN
1,0.000012,0.394,0.0521,0.43600,0.026300,0.0498,0.040400,350.480407,75.000001,408.597745,...,1.77669,173.5,15721.37910,0,1.700000e-07,0,1.666000e-07,2.720000e-25,0.0014,0.000226


In [8]:
LUX=multicurvefit()
LUX.read_json('lux2016.json')
df['lux_SI']=LUX(df.MH0,verbose=False)